In [ ]:
!pip install accelerate
!pip install sentencepiece

!pip install torch
!pip install transformers==4.45
!pip install guidance # work with transformers==4.45
!pip install huggingface_hub

!pip install gpustat

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
from huggingface_hub import login
login(os.getenv("HF_ACCESS_TOKEN"))


In [81]:
from guidance import models
gpt = models.Transformers('google/gemma-2-2b-it', device='cuda')

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

TypeError: __init__() got an unexpected keyword argument 'device'

In [59]:
from jinja2 import Environment, FileSystemLoader
def load_template(template_dir, template_name):
    # Create a Jinja environment
    env = Environment(loader=FileSystemLoader(template_dir))

    # Load a specific template
    template = env.get_template(template_name)

    # Print the output
    return template

instruction =  "Remove points of view:" 
doc = """the term ""deposit of faith"" refers to the entirety of jesus christ's divine revelation, and is passed forward to successive generations in two different forms, sacred scripture (the bible) and sacred tradition (apostolic succession)."""

temp = load_template("/home/piyush.sar/Projects/LegalSifter/llm-edit/src/templates", "prompt simplified.jinja")

prompt = temp.render(text=doc, instructions=[{"id": 1, "task": "Remove points of view:"}])

## NER Example

In [35]:
# EXAMPLE NER
import guidance
@guidance(stateless=True)
def ner_instruction(lm, input):
    lm += f'''\
    Please tag each word in the input with PER, ORG, LOC, or nothing
    ---
    Input: John worked at Apple.
    Output:
    John: PER
    worked: 
    at: 
    Apple: ORG
    .: 
    ---
    Input: {input}
    Output:
    '''
    return lm
input = 'Julia never went to Morocco in her life!!'
gpt + ner_instruction(input) + gen(stop='---')

Failed to get the top k probabilities for the tokens [2, 5958, 5886, 1853, 2204, 575, 573, 3772, 675, 12504, 235269, 153235, 235269, 25689, 235269, 689, 4285, 108, 3976, 108, 3678, 235292, 3350, 6800, 696, 9865, 235265, 108, 6140, 235292, 108, 6726, 235292, 12504, 108, 64823, 235292, 235248, 108, 482, 235292, 235248, 108, 22732, 235292, 153235, 108, 5672, 235248, 108, 3976, 108, 3678, 235292, 31918, 2447, 3815, 577, 52237, 575, 1070, 1913, 2024, 108, 6140, 235292, 108, 53270, 235292, 25689, 108, 29422, 235292, 153235, 108, 36626, 235292, 235248, 108, 511, 235292, 235248, 108, 147245, 235292, 25689, 108, 473, 235292, 235248, 108, 2794, 235292, 25689, 108]. Error: 'int' object is not iterable


In [36]:
# EXAMPLE NER
import re

@guidance(stateless=True)
def constrained_ner(lm, input):
    # Split into words
    words = [x for x in re.split('([^a-zA-Z0-9])', input) if x and not re.match('\s', x)]
    ret = ''
    for x in words:
        ret += x + ': ' + select(['PER', 'ORG', 'LOC', '']) + '\n'
    return lm + ret
gpt + ner_instruction(input) + constrained_ner(input)

Failed to get the top k probabilities for the tokens [2, 5958, 5886, 1853, 2204, 575, 573, 3772, 675, 12504, 235269, 153235, 235269, 25689, 235269, 689, 4285, 108, 3976, 108, 3678, 235292, 3350, 6800, 696, 9865, 235265, 108, 6140, 235292, 108, 6726, 235292, 12504, 108, 64823, 235292, 235248, 108, 482, 235292, 235248, 108, 22732, 235292, 153235, 108, 5672, 235248, 108, 3976, 108, 3678, 235292, 31918, 2447, 3815, 577, 52237, 575, 1070, 1913, 2024, 108, 6140, 235292, 108, 53270, 235292, 25689, 108, 29422, 235292, 153235, 108, 36626, 235292, 235248, 108, 511, 235292, 235248, 108, 147245, 235292, 25689, 108, 473, 235292, 235248, 108, 2794, 235292, 25689, 108, 7641, 235292, 153235, 108, 39872, 235248, 108, 39872, 235248, 108]. Error: 'int' object is not iterable


## Grammer for insert

In [39]:
from guidance import gen, select

In [47]:
# example
@guidance(stateless=False)
def test(lm):
    lm += 'Should I say "Scott"?\n' + select(['yes', 'no'], name='answer') + '\n'
    if lm['answer'] == 'yes':
        lm += 'Scott'
    else:
        lm += 'Not Scott'
    return lm
gpt + test()

Failed to get the top k probabilities for the tokens [2, 15284, 590, 1931, 664, 23978, 27017, 108, 3276, 108, 23978]. Error: 'int' object is not iterable


In [80]:

words = doc.split()

@guidance(stateless=False)
def constrained_ner(lm, words):
    # Split into words
    for i,x in enumerate(words):
        op = select(['<insert>', x],name=x)
        lm += op
        if lm[x] == '<insert>':
            lm += gen(max_tokens=100, stop='</insert>')
            lm += '</insert>'
            lm += x
        else:
            if i == len(words) - 1:
                lm += '<insert>'
                lm += gen(max_tokens=100)
                lm += '</insert>'
            
        lm += ' '    
    return lm

gpt + prompt + constrained_ner(words)



Failed to get the top k probabilities for the tokens [2, 5559, 476, 3110, 578, 20085, 3797, 576, 573, 2764, 2793, 235265, 5362, 573, 2412, 17923, 604, 62791, 235292, 109, 235274, 235265, 5231, 2341, 1622, 13193, 95573, 5362, 100456, 8831, 235307, 32403, 1404, 3381, 77595, 8831, 65529, 577, 1843, 8944, 689, 5942, 26099, 1570, 11969, 235265, 1699, 5566, 235292, 108, 140, 235290, 100456, 8831, 235313, 1636, 3287, 235269, 16036, 2003, 235269, 12150, 476, 24748, 11652, 235269, 578, 39017, 23576, 16512, 708, 7017, 5742, 577, 4771, 24539, 7221, 8831, 65529, 108, 140, 108, 235284, 235265, 5231, 13681, 58368, 13193, 95573, 5362, 100456, 8831, 32403, 3312, 577, 6504, 77595, 4958, 65529, 577, 8958, 35070, 689, 26099, 1570, 10626, 235265, 1699, 3287, 235292, 108, 140, 235290, 100456, 4958, 235313, 8311, 604, 8691, 1578, 235290, 16978, 727, 4958, 65529, 109, 235304, 235265, 5231, 32928, 13193, 95573, 69196, 5231, 4958, 688, 578, 5231, 8831, 688, 16323, 577, 7919, 476, 20911, 675, 978, 11352, 2113, 